## Pengenalan

Pelajaran ini akan merangkumi:
- Apa itu pemanggilan fungsi dan kegunaannya
- Cara membuat pemanggilan fungsi menggunakan Azure OpenAI
- Cara mengintegrasikan pemanggilan fungsi ke dalam aplikasi

## Matlamat Pembelajaran

Selepas melengkapkan pelajaran ini, anda akan tahu cara dan memahami:

- Tujuan menggunakan pemanggilan fungsi
- Menyediakan Pemanggilan Fungsi menggunakan Azure Open AI Service
- Merangka pemanggilan fungsi yang berkesan untuk kegunaan aplikasi anda


## Memahami Panggilan Fungsi

Untuk pelajaran ini, kita ingin membina satu ciri untuk syarikat permulaan pendidikan kita yang membolehkan pengguna menggunakan chatbot untuk mencari kursus teknikal. Kami akan mencadangkan kursus yang sesuai dengan tahap kemahiran mereka, peranan semasa dan teknologi yang diminati.

Untuk menyiapkan ini, kita akan menggunakan gabungan:
 - `Azure Open AI` untuk mencipta pengalaman chat untuk pengguna
 - `Microsoft Learn Catalog API` untuk membantu pengguna mencari kursus berdasarkan permintaan mereka
 - `Function Calling` untuk mengambil pertanyaan pengguna dan menghantarnya ke fungsi bagi membuat permintaan API.

Untuk bermula, mari kita lihat mengapa kita ingin menggunakan function calling pada mulanya:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dapatkan respons baru daripada GPT di mana ia boleh melihat respons fungsi


print(second_response.choices[0].message)


### Kenapa Perlu Function Calling

Jika anda telah menyelesaikan mana-mana pelajaran lain dalam kursus ini, anda mungkin sudah faham kehebatan menggunakan Large Language Models (LLMs). Diharapkan anda juga dapat melihat beberapa kekurangannya.

Function Calling ialah satu ciri dalam Azure Open AI Service untuk mengatasi kekangan berikut:
1) Format respons yang konsisten
2) Keupayaan menggunakan data dari sumber lain dalam aplikasi dalam konteks perbualan

Sebelum adanya function calling, respons daripada LLM adalah tidak berstruktur dan tidak konsisten. Pembangun perlu menulis kod pengesahan yang rumit untuk memastikan mereka boleh mengendalikan setiap variasi respons.

Pengguna tidak boleh mendapatkan jawapan seperti "Apakah cuaca semasa di Stockholm?". Ini kerana model-model ini terhad kepada masa data dilatih sahaja.

Mari kita lihat contoh di bawah yang menerangkan masalah ini:

Katakan kita ingin membina satu pangkalan data maklumat pelajar supaya kita boleh mencadangkan kursus yang sesuai kepada mereka. Di bawah ini terdapat dua penerangan pelajar yang sangat serupa dari segi data yang terkandung.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Kami ingin menghantar ini kepada LLM untuk menganalisis data. Ini boleh digunakan kemudian dalam aplikasi kami untuk dihantar ke API atau disimpan dalam pangkalan data.

Mari kita cipta dua arahan yang sama yang akan kita arahkan kepada LLM tentang maklumat apa yang kami minati:


Kami ingin menghantar ini kepada LLM untuk menganalisis bahagian yang penting untuk produk kami. Jadi kami boleh mencipta dua arahan yang sama untuk mengarahkan LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Selepas mencipta dua arahan ini, kita akan menghantarnya kepada LLM dengan menggunakan `openai.ChatCompletion`. Kita menyimpan arahan tersebut dalam pembolehubah `messages` dan menetapkan peranan kepada `user`. Ini adalah untuk meniru mesej daripada pengguna yang ditulis kepada chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Sekarang kita boleh menghantar kedua-dua permintaan kepada LLM dan memeriksa respons yang kita terima.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Walaupun arahan yang diberikan sama dan penerangannya serupa, kita boleh dapatkan format yang berbeza untuk sifat `Grades`.

Jika anda jalankan sel di atas beberapa kali, formatnya boleh jadi `3.7` atau `3.7 GPA`.

Ini kerana LLM mengambil data tidak berstruktur dalam bentuk arahan bertulis dan juga memulangkan data yang tidak berstruktur. Kita perlu ada format berstruktur supaya kita tahu apa yang dijangka apabila menyimpan atau menggunakan data ini.

Dengan menggunakan pemanggilan fungsi, kita boleh pastikan kita menerima data yang berstruktur. Apabila menggunakan pemanggilan fungsi, LLM sebenarnya tidak memanggil atau menjalankan sebarang fungsi. Sebaliknya, kita cipta satu struktur untuk LLM ikut dalam responsnya. Kemudian, kita gunakan respons berstruktur itu untuk tahu fungsi mana yang perlu dijalankan dalam aplikasi kita.


![Rajah Aliran Pemanggilan Fungsi](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ms.png)


### Kes Penggunaan untuk menggunakan panggilan fungsi

**Memanggil Alat Luaran**  
Chatbot sangat bagus untuk memberikan jawapan kepada soalan daripada pengguna. Dengan menggunakan panggilan fungsi, chatbot boleh menggunakan mesej daripada pengguna untuk melengkapkan tugasan tertentu. Contohnya, seorang pelajar boleh meminta chatbot untuk "Hantar emel kepada pensyarah saya mengatakan saya perlukan lebih bantuan dengan subjek ini". Ini boleh membuat panggilan fungsi kepada `send_email(to: string, body: string)`

**Membuat Permintaan API atau Pangkalan Data**  
Pengguna boleh mencari maklumat menggunakan bahasa semula jadi yang akan ditukar kepada permintaan API atau pertanyaan yang diformatkan. Contohnya, seorang guru yang meminta "Siapa pelajar yang telah menyiapkan tugasan terakhir" yang boleh memanggil fungsi bernama `get_completed(student_name: string, assignment: int, current_status: string)`

**Mewujudkan Data Berstruktur**  
Pengguna boleh mengambil satu blok teks atau CSV dan menggunakan LLM untuk mengekstrak maklumat penting daripadanya. Contohnya, seorang pelajar boleh menukar artikel Wikipedia tentang perjanjian damai untuk mencipta kad imbas AI. Ini boleh dilakukan dengan menggunakan fungsi yang dipanggil `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Membuat Panggilan Fungsi Pertama Anda

Proses untuk membuat panggilan fungsi melibatkan 3 langkah utama:
1. Memanggil API Chat Completions dengan senarai fungsi anda dan mesej daripada pengguna
2. Membaca respons model untuk melakukan tindakan seperti menjalankan fungsi atau panggilan API
3. Membuat panggilan semula ke API Chat Completions dengan respons daripada fungsi anda untuk menggunakan maklumat tersebut bagi menghasilkan jawapan kepada pengguna.


![Aliran Panggilan Fungsi](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ms.png)


### Elemen dalam panggilan fungsi

#### Input Pengguna

Langkah pertama adalah mencipta mesej pengguna. Ini boleh diberikan secara dinamik dengan mengambil nilai dari input teks atau anda boleh tetapkan nilai di sini. Jika ini kali pertama anda menggunakan API Chat Completions, kita perlu tetapkan `role` dan `content` untuk mesej tersebut.

`role` boleh jadi sama ada `system` (mencipta peraturan), `assistant` (model) atau `user` (pengguna akhir). Untuk panggilan fungsi, kita akan tetapkan ini sebagai `user` dan berikan contoh soalan.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Membuat fungsi.

Seterusnya kita akan tetapkan satu fungsi dan parameter untuk fungsi tersebut. Kita akan gunakan satu fungsi sahaja di sini iaitu `search_courses` tetapi anda boleh cipta beberapa fungsi lain.

**Penting** : Fungsi akan dimasukkan dalam mesej sistem kepada LLM dan akan mengambil sebahagian daripada jumlah token yang anda ada.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisi**

`name` - Nama fungsi yang ingin kita panggil.

`description` - Ini adalah penerangan tentang bagaimana fungsi tersebut berfungsi. Di sini penting untuk jelaskan dengan terperinci dan jelas.

`parameters` - Senarai nilai dan format yang anda mahu model hasilkan dalam responsnya.

`type` - Jenis data bagi sifat-sifat yang akan disimpan.

`properties` - Senarai nilai khusus yang akan digunakan oleh model untuk responsnya.

`name` - Nama sifat yang akan digunakan oleh model dalam respons yang diformat.

`type` - Jenis data bagi sifat ini.

`description` - Penerangan tentang sifat tertentu ini.

**Pilihan**

`required` - sifat yang diperlukan supaya panggilan fungsi dapat diselesaikan.


### Membuat panggilan fungsi
Selepas mendefinisikan fungsi, kini kita perlu memasukkannya dalam panggilan ke Chat Completion API. Kita lakukan ini dengan menambah `functions` ke dalam permintaan. Dalam kes ini `functions=functions`.

Terdapat juga pilihan untuk menetapkan `function_call` kepada `auto`. Ini bermaksud kita akan membiarkan LLM menentukan fungsi mana yang patut dipanggil berdasarkan mesej pengguna dan bukannya kita tetapkan sendiri.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sekarang mari kita lihat respons dan bagaimana ia diformat:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Anda boleh lihat bahawa nama fungsi dipanggil dan daripada mesej pengguna, LLM dapat mencari data untuk dimasukkan ke dalam argumen fungsi.


## 3. Mengintegrasikan Panggilan Fungsi ke dalam Aplikasi.

Selepas kita menguji respons yang telah diformat daripada LLM, kini kita boleh mengintegrasikannya ke dalam aplikasi.

### Mengurus aliran

Untuk mengintegrasikannya ke dalam aplikasi kita, mari ikuti langkah-langkah berikut:

Pertama, buat panggilan ke perkhidmatan Open AI dan simpan mesej tersebut dalam pembolehubah yang dipanggil `response_message`.


In [ ]:
response_message = response.choices[0].message

Sekarang kita akan mentakrifkan fungsi yang akan memanggil API Microsoft Learn untuk mendapatkan senarai kursus:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Sebagai amalan terbaik, kita akan lihat sama ada model ingin memanggil sesuatu fungsi. Selepas itu, kita akan cipta salah satu fungsi yang tersedia dan padankan dengan fungsi yang sedang dipanggil. 
Kemudian, kita akan ambil argumen fungsi tersebut dan padankan dengan argumen daripada LLM.

Akhir sekali, kita akan lampirkan mesej panggilan fungsi dan nilai yang dikembalikan oleh mesej `search_courses`. Ini akan memberikan semua maklumat yang diperlukan oleh LLM
untuk membalas kepada pengguna menggunakan bahasa semula jadi.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Cabaran Kod

Kerja yang bagus! Untuk meneruskan pembelajaran anda tentang Azure Open AI Function Calling, anda boleh bina: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Lebih banyak parameter fungsi yang mungkin membantu pelajar mencari lebih banyak kursus. Anda boleh lihat parameter API yang tersedia di sini:
 - Cipta satu lagi panggilan fungsi yang mengambil lebih banyak maklumat daripada pelajar seperti bahasa ibunda mereka
 - Cipta pengendalian ralat apabila panggilan fungsi dan/atau panggilan API tidak memulangkan sebarang kursus yang sesuai



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sah. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
